## Create CSV file from the XML files

In [65]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
import tensorflow as tf
from PIL import Image
import io
import sys

sys.path.append('./Tensorflow_object_detection/research/object_detection/')
sys.path.append('./Tensorflow_object_detection/research/')

from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

In [4]:
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    image_path = os.path.join(os.getcwd(), 'annotations')
    xml_df = xml_to_csv(image_path)
    xml_df.to_csv('data/wrench_labels.csv', index=None)
    print('Successfully converted xml to csv.')

In [5]:
main()

Successfully converted xml to csv.


## Split the labels into train and test

In [19]:
full_labels = pd.read_csv('data/wrench_labels.csv')

In [20]:
gb = full_labels.groupby('filename')
grouped_list = [gb.get_group(x) for x in gb.groups]
nb_images = len(grouped_list)

In [21]:
train_index = np.random.choice(nb_images, size=int(nb_images*0.8), replace=False)

In [26]:
test_index = np.setdiff1d(list(range(nb_images)), train_index)

In [39]:
len(train_index), len(test_index), nb_images

(371, 93, 464)

In [40]:
train = pd.concat([grouped_list[i] for i in train_index])
test = pd.concat([grouped_list[i] for i in test_index])

In [41]:
train.to_csv('data/train_labels.csv', index=None)
test.to_csv('data/test_labels.csv', index=None)

## Create the TFrecord files

In [43]:
def class_text_to_int(row_label):
    if row_label == 'top':
        return 1
    elif row_label == 'bottom':
        return 2
    else:
        None

In [44]:
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


In [45]:
def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [68]:
def main(output_path, csv_input):
    writer = tf.python_io.TFRecordWriter(output_path)
    path = os.path.join(os.getcwd(), 'images')
    examples = pd.read_csv(csv_input)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))

In [69]:
main('data/train.record','data/train_labels.csv')
main('data/test.record','data/test_labels.csv')

Successfully created the TFRecords: /Users/jesperchristensen/Dropbox/MSc/semester_3/02456_DeepLearning/Project/Tensorflow_object_detection/data/train.record
Successfully created the TFRecords: /Users/jesperchristensen/Dropbox/MSc/semester_3/02456_DeepLearning/Project/Tensorflow_object_detection/data/test.record
